### **Lab 2: Introduction to Complex Exponentials**

The goal of this laboratory is to gain familiarity with complex numbers and their use in representing sinusoidal signals as complex exponentials.

**Complex Numbers in Python**


Python can be used to compute complex-valued formulas and also to display the results as a vector or “phasor” diagrams.

Here are some of numpy package complex number functions (remember to import package):

*conj ()*       Complex conjugate

*abs ()*        Magnitude

*angle ()*      Angle (or phase) in radians

*real ()*       Real part

*imag ()*       Imaginary part
*j *        pre-defined as <math>&radic;-1</math>

*x = 3 + 4j*    j sufix defines imaginary constant

exp(1j<math>*</math>theta)  Function for the complex exponential

Each of these functions takes a vector (or matrix) as its input argument and operates on each element of the vector.

To display a complex number as a point in the complex plane you can directly use the provided function `plot_complex`. It can take a single number or a list of complex numbers as input. For instance, to display the complex number 2+1j:

In [1]:
import numpy as np
from IPython.display import Audio

from util import load_audio, save_audio, plot_signals, plot_complex

In [2]:
z = 2 + 1j
plot_complex(z)

And for displaying several complex numbers:

In [3]:
z1 = 2 + 1j
z2 = 1j
z3 = -0.5j
plot_complex([z1, z2, z3], name=['z1', 'z2', 'z3'])

# **Exercises**

**1. Complex Numbers**

To exercise your understanding of complex numbers, do the following:

1.1. Define $z_1 = -1+j0.3$ and $z_2 = 0.8+j0.7$. Enter these in Python and plot them as points and vectors in the complex plane.

In [4]:
z1 = -1 + 0.3j
z2 = 0.8 + 0.7j
plot_complex([z1, z2], name=['z1','z2'])

1.2. Compute the conjugate z* and the inverse 1/z for both $z_1$ and $z_2$ and plot the results as vectors in the complex plane.

In [5]:
zc1 = z1.conjugate()
zc2 = z2.conjugate()
plot_complex([zc1, zc2], name=["z1*","z2*"])

**2. Complex Exponentials**

Now let's work with complex exponentials. In python is very easy to work with these type of signals:

In [6]:
A = 1
f0 = 200
fs = 44100
phi = np.pi/2
t = np.arange(0, .1, 1.0/fs)
x = A * np.exp(1j*(2*f0*np.pi*t + phi))

Now we can plot the real and imaginary part of this signal:


In [7]:
plot_signals([np.real(x), np.imag(x)], fs, name=['real part', 'imag part'])

2.1. Define a complex exponential with the same parameters that those from Lab 1 (Ex 3.1) and plot the real part.

In [8]:
import numpy as np
A = 0.7635639
f0 = 531.345
phi = np.pi*6/5
fs = 44100
t = np.arange(0, 0.2, 1/fs)
x = A * np.exp(1j*(2*f0*np.pi*t + phi))

plot_signals([np.real(x)], fs, name=['real part'])

**3. Harmonic signals**

Now, we will work with harmonic signals. Until now, we have been working with simple sinusoids signals but most musical instruments sounds are harmonic. This means that they are formed by a sinusoid of the fundamental frequency plus sinusoids with frequencies multiples of it. For instance, we can define the following signal formed by the fundamental frequency plus the second and the third harmonic (note that each wave has a different phase).



3.1. Load your reference audio signal and plot some periods (5-10) where the amplitude is stable. For instance see Ex. 2.3 from Lab 1.

In [9]:
# Write the code here
filepath = "celtic-harp-c5.wav"
ref, fs = load_audio(filepath)
Audio(ref, rate=fs)
plot_signals(ref, fs, t_start=0.19, t_end=0.205056)


3.2. Now, define a harmonic signal, `y` whose fundamental frequency is the defined in Lab 1. Go step by step adding a new harmonic in each step. Plot both signals (the reference and the synthesized) and try to reproduce the shape of the reference signal.

**Note 1**: in order to have a similar shape, we need to select the amplitudes and phases carefully. One way to do this is to define the harmonic signal as follows:

$$y(t) = \sum_{k=1}^K A_k\cos\left(2\pi kf_0 t + k \phi - (k-1)\pi/2 \right), $$

where $K$ is the number of harmonics, $f_0$ is the fundamental frequency, $A_k$ is the amplitude (weight) of each harmonic and $\phi$ is the phase of the signal (defined in Lab1 Ex 2.4).

**Note 2**:
Define the $A_k$ values relative to the fundamental frequency. This means to define $A_1=1$ and the others less than 1. You can use Audacity to plot the spectrum of the fragment selected of the reference audio and measure the relative amplitudes of the harmonics.

**Note 3**: Normalize the amplitude of the signal by the same amplitude of the reference. For instance, if the amplitude of the reference signal is 0.33, you can normalize the syntesized signal by first dividing by its maximum and then multiplyng by 0.33:

```
y = 0.33 * y / np.amax(y)
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_har = 20                    
T = 0.001882
f0 = 1 / T
t_start, t_end = 0.19, 0.205056
i_start, i_end = int(t_start * fs), int(t_end * fs)
segment = ref[i_start:i_end]
N = len(segment)
t_seg = np.arange(N) / fs
Y = np.fft.fft(segment)
freqs = np.fft.fftfreq(N, 1/fs)

mask = freqs > 0 #Selecting only positive frequencies
freqs = freqs[mask]
Y = Y[mask]

amplitudes, phases = [], []
for k in range(1, num_har + 1):
    idx = np.argmin(np.abs(freqs - k * f0))
    amplitudes.append(2 * np.abs(Y[idx]) / N)
    phases.append(np.angle(Y[idx]))

amplitudes, phases = np.array(amplitudes), np.array(phases)

t = np.arange(0, len(ref)/fs, 1/fs) 

y = sum(A * np.cos(2*np.pi*k*f0*t + phi)for k, (A, phi) in enumerate(zip(amplitudes, phases), start=1))
y *= np.max(np.abs(segment)) / np.max(np.abs(y))

plot_signals([segment, y], fs, t_start=0, t_end=t_seg[-1], name=['Reference', 'Synthesized'])

3.3. Listen to the synthtesis and remark what are the main differences between the reference and synthesis.

In [11]:
save_audio('synthesised_wave_lab2.wav', y, fs)
Audio(y, rate=fs)

The synthesized signal sounds artificial compared to the reference. While the tone is accurate, the sound is duller because it lacks all the very high-pitched frequency content and noise present in the original signal. Crucially, because the synthesis uses constant values, it lacks the natural amplitude and frequency variations over time that make the original sound feel dynamic and real. However, this 20-harmonic version sounds much fuller and more realistic than the single-sine wave generated in Lab1.